# Data Preparation

In [3]:
# common imports
import requests
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import sys
from datetime import datetime

sys.executable

'/opt/anaconda3/bin/python'

In [4]:
sys.path

['/Users/mirror/Desktop/GitHub/NFL_Game_Predictor',
 '/opt/anaconda3/lib/python38.zip',
 '/opt/anaconda3/lib/python3.8',
 '/opt/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/Users/mirror/.local/lib/python3.8/site-packages',
 '/opt/anaconda3/lib/python3.8/site-packages',
 '/opt/anaconda3/lib/python3.8/site-packages/aeosa',
 '/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions',
 '/Users/mirror/.ipython']

In [5]:
# Importing data from csv file
path = "/Users/mirror/Desktop/GitHub/nfl_game_predictor/data/scraped_data.csv"
df = pd.read_csv(path)

### Quick Exploration

In [6]:
df.shape

(15000, 26)

In [7]:
df.dtypes

season                 int64
team                  object
week                  object
day                   object
date                  object
time                  object
result                object
ot                    object
record                object
home_team             object
opp                   object
points_scored        float64
points_allowed       float64
1st_downs            float64
totyd                float64
passyd               float64
rushyd               float64
to                   float64
1st_downs_allowed    float64
totyd_allowed        float64
passyd_allowed       float64
rushyd_allowed       float64
to_forced            float64
off_exp_pts          float64
def_exp_pts          float64
sts_exp_pts          float64
dtype: object

In [8]:
df.isnull().sum()

season                   0
team                     0
week                     0
day                      0
date                     0
time                     0
result                   0
ot                   14078
record                   0
home_team             7472
opp                      0
points_scored            0
points_allowed           0
1st_downs                0
totyd                    0
passyd                   2
rushyd                   0
to                    3287
1st_downs_allowed        0
totyd_allowed            0
passyd_allowed           2
rushyd_allowed           0
to_forced             3287
off_exp_pts              0
def_exp_pts              0
sts_exp_pts              0
dtype: int64

In [9]:
df['ot'].isnull().sum()

14078

In [10]:
# discovering how many times an opponent played a game
df.opp.value_counts()

New England Patriots        506
Green Bay Packers           498
Pittsburgh Steelers         493
Indianapolis Colts          489
Philadelphia Eagles         487
Seattle Seahawks            485
San Francisco 49ers         484
Denver Broncos              480
Kansas City Chiefs          478
Minnesota Vikings           477
Dallas Cowboys              477
New Orleans Saints          476
Atlanta Falcons             475
New York Giants             474
Tampa Bay Buccaneers        474
New York Jets               471
Miami Dolphins              470
Arizona Cardinals           469
Chicago Bears               468
Buffalo Bills               468
Cincinnati Bengals          468
Detroit Lions               463
Carolina Panthers           458
Jacksonville Jaguars        455
Baltimore Ravens            452
Washington Redskins         423
Cleveland Browns            414
Oakland Raiders             408
Tennessee Titans            394
San Diego Chargers          382
St. Louis Rams              346
Houston 

In [11]:
# importing a new library that was recommended to me by my mentor
import sweetviz as sv

df_report = sv.analyze(df)
df_report.show_html()
#df_report.show_notebook()

/opt/anaconda3/lib/python3.8/site-packages/sweetviz/dataframe_report.py:74: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  all_source_names = [cur_name for cur_name, cur_series in source_df.iteritems()]
/opt/anaconda3/lib/python3.8/site-packages/sweetviz/dataframe_report.py:109: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  filtered_series_names_in_source = [cur_name for cur_name, cur_series in source_df.iteritems()


/opt/anaconda3/lib/python3.8/site-packages/sweetviz/series_analyzer_numeric.py:25: FutureWarning: The 'mad' method is deprecated and will be removed in a future version. To compute the same result, you may do `(df - df.mean()).abs().mean()`.
  stats["mad"] = series.mad()
/opt/anaconda3/lib/python3.8/site-packages/sweetviz/series_analyzer_cat.py:28: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item in category_counts.iteritems():
/opt/anaconda3/lib/python3.8/site-packages/sweetviz/series_analyzer_cat.py:28: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item in category_counts.iteritems():
/opt/anaconda3/lib/python3.8/site-packages/sweetviz/series_analyzer_cat.py:28: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item in category_counts.iteritems():
/opt/anaconda3/lib/python3.8/site-packages/sweetviz/series_ana


Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [12]:
# compares two dataframes via sweetviz
#df_comp = sv.compare(df)

In [13]:
# another way to get great insights is to use the comparison functionality to split your dataset into 2 sub-populations.
#df_comp_intra = sv.compare_intra(df)

### Initial data cleansing

In [14]:
# there are only supposed to be 32 teams in the NFL. some teams have changed their name and/or location.
len(df.opp.unique())

40

In [15]:
# some teams have changed their name and/or location, which created another opponent.
# manually replacing team names.
df.loc[df["opp"] == "Washington Redskins", "opp"] = "Washington Commanders"
df.loc[df["opp"] == "Washington Football Team", "opp"] = "Washington Commanders"
df.loc[df["opp"] == "Oakland Raiders", "opp"] = "Las Vegas Raiders"
df.loc[df["opp"] == "Los Angeles Raiders", "opp"] = "Las Vegas Raiders"
df.loc[df["opp"] == "Houston Oilers", "opp"] = "Tennessee Titans"
df.loc[df["opp"] == "Tennessee Oilers", "opp"] = "Tennessee Titans"
df.loc[df["opp"] == "San Diego Chargers", "opp"] = "Los Angeles Chargers"
df.loc[df["opp"] == "St. Louis Rams", "opp"] = "Los Angeles Rams"

In [16]:
# verifying that there are only 32 teams in the df.opp column now.
len(df.opp.unique())

32

### Converting Missing Values and Categorical Data Types

In [17]:
# convert misc values for columns "home/away" (home team?) and "ot" (overtime) to a binary integer
df.loc[df["home_team"] == "@", "home_team"] = 0
df.loc[df["home_team"] == "N", "home_team"] = 0
df.loc[df["ot"] == "OT", "ot"] = 1

In [18]:
# convert NaN values for columns "home/away" (home team?) to the integer 1.
df['home_team'] = df['home_team'].fillna(1)

In [19]:
# convert NaN values for columns "ot" (overtime), "to" (turnovers), and "to_forced" to the integer 0.
df['ot'] = df['ot'].fillna(0)
df['to'] = df['to'].fillna(0)
df['to_forced'] = df['to_forced'].fillna(0)

In [20]:
# convert playoff games to follow week numbers (wildcard = 19, divisional = 20, and so on... for seasons 2021).
# but remember - before 2021, teams only played 17 week regular seasons, so wildcard would be = to 18 for these years. 
# For the seasons of 2021 and beyond, teams play an 18 week season.
w = []
y = df.loc[df['season'] == 2021]
for i in y['week']:
    if i == 'Wild Card':
        w.append(19)
    elif i == 'Division':
        w.append(20)
    elif i == 'Conf. Champ.':
        w.append(21)
    elif i == 'SuperBowl':
        w.append(23)
    else:
        w.append(int(i))

dfy = df.loc[df['season'] == 2021].assign(week = w)
dfy

,season,team,week,day,date,time,result,ot,record,home_team,...,rushyd,to,1st_downs_allowed,totyd_allowed,passyd_allowed,rushyd_allowed,to_forced,off_exp_pts,def_exp_pts,sts_exp_pts
248,2021,BUF,1,Sun,September 12,1:00PM ET,L,0,0-1,1,...,117.0,1.0,16.0,252.0,177.0,75.0,0.0,-3.55,0.54,-5.01
249,2021,BUF,2,Sun,September 19,1:00PM ET,W,0,1-1,0,...,143.0,2.0,13.0,216.0,145.0,71.0,3.0,8.16,24.67,-0.37
250,2021,BUF,3,Sun,September 26,1:00PM ET,W,0,2-1,1,...,122.0,0.0,13.0,290.0,212.0,78.0,3.0,20.55,8.27,-7.29
251,2021,BUF,4,Sun,October 3,1:00PM ET,W,0,3-1,1,...,199.0,1.0,6.0,109.0,61.0,48.0,5.0,14.54,32.68,-6.40
252,2021,BUF,5,Sun,October 10,8:20PM ET,W,0,4-1,0,...,121.0,0.0,29.0,392.0,272.0,120.0,4.0,19.84,1.20,-1.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,2021,SEA,14,Sun,December 12,1:00PM ET,W,0,5-8,0,...,193.0,0.0,25.0,380.0,317.0,63.0,0.0,21.71,2.60,-9.75
814,2021,SEA,15,Tue,December 21,7:00PM ET,L,0,5-9,0,...,80.0,1.0,20.0,332.0,209.0,123.0,1.0,-8.13,-4.19,3.72
815,2021,SEA,16,Sun,December 26,4:05PM ET,L,0,5-10,1,...,170.0,0.0,26.0,317.0,229.0,88.0,0.0,8.31,-7.60,-3.95
816,2021,SEA,17,Sun,January 2,4:25PM ET,W,0,6-10,1,...,265.0,0.0,20.0,357.0,262.0,95.0,3.0,32.32,-3.55,-2.68


In [21]:
# convert playoff games to follow week numbers (wildcard = 19, divisional = 20, and so on... for seasons 2021).
# but remember - before 2021, teams only played 17 week regular seasons, so wildcard would be = to 18 for these years. 
# For the seasons of 2021 and beyond, teams play an 18 week season.
e = []
r = df.loc[df['season'] != 2021]
for i in r['week']:
    if i == 'Wild Card':
        e.append(18)
    elif i == 'Division':
        e.append(19)
    elif i == 'Conf. Champ.':
        e.append(20)
    elif i == 'SuperBowl':
        e.append(22)
    else:
        e.append(int(i))

dfe = df.loc[df['season'] != 2021].assign(week = e)
dfe

,season,team,week,day,date,time,result,ot,record,home_team,...,rushyd,to,1st_downs_allowed,totyd_allowed,passyd_allowed,rushyd_allowed,to_forced,off_exp_pts,def_exp_pts,sts_exp_pts
0,2022,BUF,1,Thu,September 8,8:20PM ET,W,0,1-0,0,...,121.0,4.0,19.0,243.0,191.0,52.0,3.0,13.89,10.29,-3.96
1,2022,BUF,2,Mon,September 19,7:15PM ET,W,0,2-0,1,...,101.0,0.0,12.0,187.0,107.0,80.0,4.0,17.69,18.01,1.55
2,2022,BUF,3,Sun,September 25,1:00PM ET,L,0,2-1,0,...,115.0,1.0,15.0,212.0,171.0,41.0,0.0,15.88,-7.45,-4.86
3,2022,BUF,4,Sun,October 2,1:00PM ET,W,0,3-1,0,...,125.0,2.0,22.0,296.0,134.0,162.0,2.0,2.10,2.66,-1.69
4,2022,BUF,5,Sun,October 9,1:00PM ET,W,0,4-1,1,...,120.0,2.0,23.0,364.0,310.0,54.0,2.0,20.66,9.42,3.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,1994,LAR,13,Sun,November 27,4:00PM ET,L,0,4-8,0,...,48.0,5.0,16.0,243.0,129.0,114.0,1.0,-15.08,12.27,-6.03
14996,1994,LAR,14,Sun,December 4,4:00PM ET,L,0,4-9,1,...,75.0,4.0,20.0,328.0,191.0,137.0,1.0,-1.17,-3.52,-13.22
14997,1994,LAR,15,Sun,December 11,1:00PM ET,L,0,4-10,0,...,63.0,2.0,17.0,355.0,230.0,125.0,0.0,-11.84,-12.38,5.25
14998,1994,LAR,16,Sun,December 18,1:00PM ET,L,0,4-11,0,...,37.0,1.0,19.0,298.0,135.0,163.0,0.0,-1.42,-6.17,-8.99


In [22]:
# combining the dataframes back together and sorting based on index.
df = pd.concat([dfy, dfe])
df = df.sort_index()

In [23]:
# game's "result" needs to be converted into an integer
df.loc[df["result"] == "W", "result"] = 1
df.loc[df["result"] == "T", "result"] = 0
df.loc[df["result"] == "L", "result"] = 0

In [24]:
# changing the data types for some of the object columns
df[["result", "home_team"]] = df[["result", "home_team"]].apply(pd.to_numeric)

In [25]:
# dropping bye week rows, playoff rows, etc.
df = df[df['result'].notna()]

In [26]:
# checking how many rows still have missing values in them
df[df.isnull().any(axis=1)]

,season,team,week,day,date,time,result,ot,record,home_team,...,rushyd,to,1st_downs_allowed,totyd_allowed,passyd_allowed,rushyd_allowed,to_forced,off_exp_pts,def_exp_pts,sts_exp_pts
6728,2009,NYJ,17,Sun,January 3,8:31PM ET,1,0,9-7,1,...,257.0,0.0,5.0,72.0,NaN,72.0,3.0,10.82,33.61,-6.11
6779,2009,CIN,17,Sun,January 3,8:31PM ET,0,0,10-6,0,...,72.0,3.0,21.0,320.0,63.0,257.0,0.0,-33.61,-10.82,6.11
10150,2003,LV,17,Sun,December 28,4:15PM ET,0,0,4-12,0,...,141.0,1.0,20.0,337.0,74.0,263.0,0.0,-25.91,-1.64,16.75
10166,2003,LAC,17,Sun,December 28,4:15PM ET,1,0,4-12,1,...,263.0,0.0,9.0,141.0,NaN,141.0,1.0,1.64,25.91,-16.75


In [27]:
# converting the last four missing values to zero. The net passing yards equaled zero due to sacks.
df['passyd'] = df['passyd'].fillna(0)
df['passyd_allowed'] = df['passyd_allowed'].fillna(0)

In [28]:
# verifying the data types in the df
df.dtypes

season                 int64
team                  object
week                   int64
day                   object
date                  object
time                  object
result                 int64
ot                     int64
record                object
home_team              int64
opp                   object
points_scored        float64
points_allowed       float64
1st_downs            float64
totyd                float64
passyd               float64
rushyd               float64
to                   float64
1st_downs_allowed    float64
totyd_allowed        float64
passyd_allowed       float64
rushyd_allowed       float64
to_forced            float64
off_exp_pts          float64
def_exp_pts          float64
sts_exp_pts          float64
dtype: object

In [29]:
# checking how many rows still have missing values in them
len(df[df.isnull().any(axis=1)])

0

### Converting Date and Time columns

In [30]:
# "day" of the week needs to be converted into an integer.
day_dict = {"Tue":1, "Wed":2, "Thu":3, "Fri":4, "Sat":5, "Sun":6, "Mon":7}

# using the manually created day_dictionary to replace the day of the week.
df = df.replace({"day_of_week": day_dict}) 

In [31]:
df['year'] = df['season'] 

In [32]:
# date need to be converted into a numerical value (month and day_of_month)
c = []
v = []
for i in df['date']:
    c.append((i.split(' ')[0]))
    v.append(int(i.split(' ')[1]))
df = df.assign(month = c)
df = df.assign(day = v)

In [33]:
# "month of the year needs to be converted into an integer. NFL games start in August (preseason) and end in February (super bowl).
month_dict = {"January":1, "February":2, "March":3, "April":4, "May":5, "June":6, 
            "July":7, "August":8, "September":9, "October":10, "November":11, "December":12}

# using the manually created day_dictionary to replace the day of the week.
df = df.replace({"month": month_dict}) 

In [34]:
# we can drop the date column after converting it
df = df.drop('date', axis=1)

In [35]:
# time needs to be converted into a numerical value
b = []
for i in df['time']:
    if 'PM' in i:
        b.append(int(i[0]) + 12)
    else:
        b.append(int(i[0]))
df = df.assign(hour = b)

In [36]:
# we can drop the time column after converting it
df = df.drop('time', axis=1)

In [37]:
# record need to be converted into a numerical value (cumulative wins and cumulative losses)
y = []
z = []
for i in df['record']:
    y.append(int(i.split('-')[0]))
    z.append(int(i.split('-')[1]))
df = df.assign(win_count = y)
df = df.assign(loss_count = z)

# we can drop the record column after converting it
df = df.drop('record', axis=1)

In [38]:
################# verify that these numerical codes correspond to the df["team"] #################
# converting opponent's team name to a nurmerical category
# we need to verify that the opp_code matches with the team_code. Each team should be labeled with identical opp and team codes.
df["opp_code"] = df["opp"].astype("category").cat.codes
df["team_code"] = df["team"].astype("category").cat.codes

In [39]:
df.loc[df['team'] == 'LAC'].head(50)

,season,team,week,day,result,ot,home_team,opp,points_scored,points_allowed,...,off_exp_pts,def_exp_pts,sts_exp_pts,year,month,hour,win_count,loss_count,opp_code,team_code
101,2022,LAC,1,11,1,0,1,Las Vegas Raiders,24.0,19.0,...,6.30,2.73,-1.95,2022,9,16,1,0,16,16
102,2022,LAC,2,15,0,0,0,Kansas City Chiefs,24.0,27.0,...,4.12,-5.54,-3.72,2022,9,20,1,1,15,16
103,2022,LAC,3,25,0,0,1,Jacksonville Jaguars,10.0,38.0,...,-10.35,-18.06,2.60,2022,9,16,1,2,14,16
104,2022,LAC,4,2,1,0,0,Houston Texans,34.0,24.0,...,16.95,-3.13,-4.62,2022,10,13,2,2,12,16
105,2022,LAC,5,9,1,0,0,Cleveland Browns,30.0,28.0,...,16.01,-16.53,4.92,2022,10,13,3,2,7,16
106,2022,LAC,6,17,1,1,1,Denver Broncos,19.0,16.0,...,-1.03,3.11,0.92,2022,10,20,4,2,9,16
107,2022,LAC,7,23,0,0,1,Seattle Seahawks,23.0,37.0,...,-4.60,-12.13,1.10,2022,10,16,4,3,28,16
499,2021,LAC,1,12,1,0,0,Washington Commanders,20.0,16.0,...,9.45,-1.45,-1.24,2021,9,13,1,0,31,16
500,2021,LAC,2,19,0,0,1,Dallas Cowboys,17.0,20.0,...,5.55,-17.50,-1.81,2021,9,16,1,1,8,16
501,2021,LAC,3,26,1,0,0,Kansas City Chiefs,30.0,24.0,...,16.65,-9.49,-3.01,2021,9,13,2,1,15,16


In [40]:
# we can drop the opp and team columns after converting it into numerical codes.
df = df.drop('opp', axis=1)
df = df.drop('team', axis=1)

In [41]:
# Reordering the columns to my preference
cols = df.columns.tolist()
cols = cols[0:1] + cols[-1:] + cols[1:3] + cols[-7:-1] + cols[3:-7]
df = df[cols]
df

,season,team_code,week,day,year,month,hour,win_count,loss_count,opp_code,...,rushyd,to,1st_downs_allowed,totyd_allowed,passyd_allowed,rushyd_allowed,to_forced,off_exp_pts,def_exp_pts,sts_exp_pts
0,2022,3,1,8,2022,9,20,1,0,18,...,121.0,4.0,19.0,243.0,191.0,52.0,3.0,13.89,10.29,-3.96
1,2022,3,2,19,2022,9,19,2,0,30,...,101.0,0.0,12.0,187.0,107.0,80.0,4.0,17.69,18.01,1.55
2,2022,3,3,25,2022,9,13,2,1,19,...,115.0,1.0,15.0,212.0,171.0,41.0,0.0,15.88,-7.45,-4.86
3,2022,3,4,2,2022,10,13,3,1,2,...,125.0,2.0,22.0,296.0,134.0,162.0,2.0,2.10,2.66,-1.69
4,2022,3,5,9,2022,10,13,4,1,26,...,120.0,2.0,23.0,364.0,310.0,54.0,2.0,20.66,9.42,3.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,1994,17,13,27,1994,11,16,4,8,17,...,48.0,5.0,16.0,243.0,129.0,114.0,1.0,-15.08,12.27,-6.03
14996,1994,17,14,4,1994,12,16,4,9,22,...,75.0,4.0,20.0,328.0,191.0,137.0,1.0,-1.17,-3.52,-13.22
14997,1994,17,15,11,1994,12,13,4,10,29,...,63.0,2.0,17.0,355.0,230.0,125.0,0.0,-11.84,-12.38,5.25
14998,1994,17,16,18,1994,12,13,4,11,5,...,37.0,1.0,19.0,298.0,135.0,163.0,0.0,-1.42,-6.17,-8.99


In [42]:
df.dtypes

season                 int64
team_code               int8
week                   int64
day                    int64
year                   int64
month                  int64
hour                   int64
win_count              int64
loss_count             int64
opp_code                int8
result                 int64
ot                     int64
home_team              int64
points_scored        float64
points_allowed       float64
1st_downs            float64
totyd                float64
passyd               float64
rushyd               float64
to                   float64
1st_downs_allowed    float64
totyd_allowed        float64
passyd_allowed       float64
rushyd_allowed       float64
to_forced            float64
off_exp_pts          float64
def_exp_pts          float64
sts_exp_pts          float64
dtype: object

In [43]:
#df['date'] =  pd.to_datetime(df['date'], format='%b%d')

In [44]:
df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])

In [45]:
df.dtypes

season                        int64
team_code                      int8
week                          int64
day                           int64
year                          int64
month                         int64
hour                          int64
win_count                     int64
loss_count                    int64
opp_code                       int8
result                        int64
ot                            int64
home_team                     int64
points_scored               float64
points_allowed              float64
1st_downs                   float64
totyd                       float64
passyd                      float64
rushyd                      float64
to                          float64
1st_downs_allowed           float64
totyd_allowed               float64
passyd_allowed              float64
rushyd_allowed              float64
to_forced                   float64
off_exp_pts                 float64
def_exp_pts                 float64
sts_exp_pts                 

In [46]:
df

,season,team_code,week,day,year,month,hour,win_count,loss_count,opp_code,...,to,1st_downs_allowed,totyd_allowed,passyd_allowed,rushyd_allowed,to_forced,off_exp_pts,def_exp_pts,sts_exp_pts,date
0,2022,3,1,8,2022,9,20,1,0,18,...,4.0,19.0,243.0,191.0,52.0,3.0,13.89,10.29,-3.96,2022-09-08 20:00:00
1,2022,3,2,19,2022,9,19,2,0,30,...,0.0,12.0,187.0,107.0,80.0,4.0,17.69,18.01,1.55,2022-09-19 19:00:00
2,2022,3,3,25,2022,9,13,2,1,19,...,1.0,15.0,212.0,171.0,41.0,0.0,15.88,-7.45,-4.86,2022-09-25 13:00:00
3,2022,3,4,2,2022,10,13,3,1,2,...,2.0,22.0,296.0,134.0,162.0,2.0,2.10,2.66,-1.69,2022-10-02 13:00:00
4,2022,3,5,9,2022,10,13,4,1,26,...,2.0,23.0,364.0,310.0,54.0,2.0,20.66,9.42,3.54,2022-10-09 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,1994,17,13,27,1994,11,16,4,8,17,...,5.0,16.0,243.0,129.0,114.0,1.0,-15.08,12.27,-6.03,1994-11-27 16:00:00
14996,1994,17,14,4,1994,12,16,4,9,22,...,4.0,20.0,328.0,191.0,137.0,1.0,-1.17,-3.52,-13.22,1994-12-04 16:00:00
14997,1994,17,15,11,1994,12,13,4,10,29,...,2.0,17.0,355.0,230.0,125.0,0.0,-11.84,-12.38,5.25,1994-12-11 13:00:00
14998,1994,17,16,18,1994,12,13,4,11,5,...,1.0,19.0,298.0,135.0,163.0,0.0,-1.42,-6.17,-8.99,1994-12-18 13:00:00


### Feature Engineering

In [47]:
# the goal is to obtain a new column that contains rolling_averages for a team over the last four games.
# should the end of a season reset the rolling percentage??? Yes
# creating a df that is organized at the team and season level
grouped_teams = df.groupby("team_code")

In [48]:
# viewing one instance
group1 = grouped_teams.get_group(1)
group1

,season,team_code,week,day,year,month,hour,win_count,loss_count,opp_code,...,to,1st_downs_allowed,totyd_allowed,passyd_allowed,rushyd_allowed,to_forced,off_exp_pts,def_exp_pts,sts_exp_pts,date
185,2022,1,1,11,2022,9,13,0,1,22,...,2.0,18.0,385.0,234.0,151.0,1.0,7.50,-4.62,0.51,2022-09-11 13:00:00
186,2022,1,2,18,2022,9,16,0,2,18,...,3.0,24.0,337.0,272.0,65.0,3.0,-3.49,-8.46,8.11,2022-09-18 16:00:00
187,2022,1,3,25,2022,9,16,1,2,28,...,2.0,23.0,420.0,308.0,112.0,1.0,20.84,-15.09,1.20,2022-09-25 16:00:00
188,2022,1,4,2,2022,10,13,2,2,7,...,1.0,22.0,403.0,226.0,177.0,2.0,5.92,-5.25,1.44,2022-10-02 13:00:00
189,2022,1,5,9,2022,10,13,2,3,29,...,0.0,26.0,420.0,351.0,69.0,0.0,3.90,-13.30,1.34,2022-10-09 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14979,1994,1,13,27,1994,11,13,6,6,25,...,3.0,14.0,318.0,203.0,115.0,2.0,-9.04,12.82,-2.86,1994-11-27 13:00:00
14980,1994,1,14,4,1994,12,16,6,7,27,...,5.0,28.0,476.0,315.0,161.0,1.0,-16.65,-15.35,1.22,1994-12-04 16:00:00
14981,1994,1,15,11,1994,12,20,6,8,22,...,1.0,19.0,402.0,264.0,138.0,0.0,9.09,-13.55,-3.68,1994-12-11 20:00:00
14982,1994,1,16,18,1994,12,13,6,9,11,...,2.0,23.0,384.0,310.0,74.0,1.0,4.46,-9.31,2.71,1994-12-18 13:00:00


In [49]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(4, closed='left').mean() # closed='left' inside rolling parameters
    # the problem is that it is using the fourth games' info to predict the fourth game and beyond.
    # closed='left' would solve this problem, but that requires datetime. How do I resolve this???
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [50]:
cols = ["result", "points_scored", "points_allowed",
        "totyd", "to", "totyd_allowed", "to_forced",
        "off_exp_pts", "def_exp_pts", "sts_exp_pts"]

new_cols = [f"{c}_rolling" for c in cols]

In [51]:
# first four weeks are being dropped because of na values from rolling(4)
# should we bring datetime back??
rolling_averages(group1, cols, new_cols)

,season,team_code,week,day,year,month,hour,win_count,loss_count,opp_code,...,result_rolling,points_scored_rolling,points_allowed_rolling,totyd_rolling,to_rolling,totyd_allowed_rolling,to_forced_rolling,off_exp_pts_rolling,def_exp_pts_rolling,sts_exp_pts_rolling
14972,1994,1,5,2,1994,10,16,3,2,18,...,0.50,24.00,23.50,377.50,2.25,388.50,3.50,4.2750,2.3575,-4.1050
14973,1994,1,6,9,1994,10,13,4,2,29,...,0.75,19.00,17.00,371.75,2.50,362.00,3.75,1.6875,6.7875,-3.5875
14974,1994,1,7,16,1994,10,13,4,3,27,...,0.75,19.75,17.00,353.75,2.75,336.75,3.50,-3.7350,10.0575,-2.4250
14975,1994,1,8,23,1994,10,16,4,4,16,...,0.75,18.00,20.00,321.25,2.75,300.25,2.50,-10.8675,8.0200,-2.3575
14976,1994,1,10,6,1994,11,13,5,4,17,...,0.50,15.50,22.50,307.75,3.00,290.00,1.50,-9.5000,6.0425,-3.0400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,2022,1,4,2,2022,10,13,2,2,7,...,0.50,25.00,24.25,329.25,2.00,370.00,1.50,7.5675,-8.6825,3.0125
189,2022,1,5,9,2022,10,13,2,3,29,...,0.50,25.75,25.25,349.00,2.00,386.25,1.75,7.6925,-8.3550,2.8150
190,2022,1,6,16,2022,10,13,3,3,27,...,0.50,23.00,23.75,310.25,1.50,395.00,1.50,6.7925,-10.5250,3.0225
191,2022,1,7,23,2022,10,13,3,4,6,...,0.75,23.25,19.50,317.25,0.75,397.25,1.50,10.5550,-8.5575,0.5725


In [52]:
#apply the new rolling_averages columns to each group
df_rolling = df.groupby(["team_code", "season"]).apply(lambda x: rolling_averages(x, cols, new_cols))

In [53]:
df_rolling

season  team_code  week  day  year  month  hour  \
team_code season                                                          
0         1994   14803    1994          0     6    9  1994     10    16   
                 14804    1994          0     7   16  1994     10    13   
                 14805    1994          0     8   23  1994     10    16   
                 14806    1994          0     9   30  1994     10    20   
                 14807    1994          0    10    6  1994     11    16   
...                        ...        ...   ...  ...   ...    ...   ...   
31        2021   583      2021         31    16   26  2021     12    20   
          2022   151      2022         31     5    9  2022     10    13   
                 152      2022         31     6   13  2022     10    20   
                 153      2022         31     7   23  2022     10    13   
                 154      2022         31     8   30  2022     10    16   

                        win_count  loss_count  opp_code  ...  result_rolling  \
team_code season                                         ...                   
0         1994   14803          1           4         8  ...            0.25   
                 14804          2           4        31  ...            0.25   
                 14805          2           5         8  ...            0.50   
                 14806          3           5        26  ...            0.50   
                 14807          3           6        25  ...            0.50   
...                           ...         ...       ...  ...             ...   
31        2021   583            6           9         8  ...            0.50   
          2022   151            1           4        30  ...            0.25   
                 152            2           4         5  ...            0.00   
                 153            3           4        11  ...            0.25   
                 154            4           4        13  ...            0.50   

                        points_scored_rolling  points_allowed_rolling  \
team_code season                                                        
0         1994   14803                  11.50                   18.25   
                 14804                   9.25                   24.25   
                 14805                   9.75                   23.25   
                 14806                  15.00                   22.25   
                 14807                  15.75                   24.75   
...                                       ...                     ...   
31        2021   583                    17.75                   21.00   
          2022   151                    18.25                   26.75   
                 152                    15.50                   26.50   
                 153                    11.75                   19.25   
                 154                    15.50                   18.50   

                        totyd_rolling  to_rolling  totyd_allowed_rolling  \
team_code season                                                           
0         1994   14803         258.75        2.75                 259.50   
                 14804         255.50        3.25                 309.25   
                 14805         293.00        3.25                 316.25   
                 14806         292.25        2.50                 313.75   
                 14807         298.75        2.25                 303.50   
...                               ...         ...                    ...   
31        2021   583           282.50        1.50                 354.75   
          2022   151           330.75        1.75                 371.75   
                 152           329.50        1.25                 336.25   
                 153           284.00        1.00                 327.75   
                 154           315.00        1.00                 285.75   

                        to_forced_rolling  off_exp_pts_rolling  \
te

In [54]:
# dropping extra index levels
df_rolling = df_rolling.droplevel('team_code')
df_rolling = df_rolling.droplevel('season')
df_rolling

,season,team_code,week,day,year,month,hour,win_count,loss_count,opp_code,...,result_rolling,points_scored_rolling,points_allowed_rolling,totyd_rolling,to_rolling,totyd_allowed_rolling,to_forced_rolling,off_exp_pts_rolling,def_exp_pts_rolling,sts_exp_pts_rolling
14803,1994,0,6,9,1994,10,16,1,4,8,...,0.25,11.50,18.25,258.75,2.75,259.50,2.50,-14.1225,7.6750,2.9600
14804,1994,0,7,16,1994,10,13,2,4,31,...,0.25,9.25,24.25,255.50,3.25,309.25,2.00,-16.9475,0.2700,3.9425
14805,1994,0,8,23,1994,10,16,2,5,8,...,0.50,9.75,23.25,293.00,3.25,316.25,2.75,-17.5550,7.3875,1.8750
14806,1994,0,9,30,1994,10,20,3,5,26,...,0.50,15.00,22.25,292.25,2.50,313.75,2.25,-10.5900,5.3700,2.3050
14807,1994,0,10,6,1994,11,16,3,6,25,...,0.50,15.75,24.75,298.75,2.25,303.50,2.00,-10.1350,3.6550,0.7575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,2021,31,16,26,2021,12,20,6,9,8,...,0.50,17.75,21.00,282.50,1.50,354.75,1.25,-2.9050,-1.2000,1.2875
151,2022,31,5,9,2022,10,13,1,4,30,...,0.25,18.25,26.75,330.75,1.75,371.75,0.25,-4.0950,-6.1525,0.5475
152,2022,31,6,13,2022,10,20,2,4,5,...,0.00,15.50,26.50,329.50,1.25,336.25,0.00,-6.2225,-5.3050,-0.4250
153,2022,31,7,23,2022,10,13,3,4,11,...,0.25,11.75,19.25,284.00,1.00,327.75,0.50,-9.4900,-2.9675,3.7075


In [55]:
# resetting the index for the df
df_rolling.index = range(df_rolling.shape[0])
df_rolling.tail(50)

,season,team_code,week,day,year,month,hour,win_count,loss_count,opp_code,...,result_rolling,points_scored_rolling,points_allowed_rolling,totyd_rolling,to_rolling,totyd_allowed_rolling,to_forced_rolling,off_exp_pts_rolling,def_exp_pts_rolling,sts_exp_pts_rolling
11298,2018,31,10,11,2018,11,13,6,3,29,...,0.75,19.25,21.25,329.75,0.50,366.75,2.00,2.0100,-7.6600,5.0150
11299,2018,31,11,18,2018,11,13,6,4,12,...,0.75,17.50,17.75,329.25,0.50,404.50,2.25,2.1575,-6.4825,4.5575
11300,2018,31,12,22,2018,11,16,6,5,8,...,0.50,17.75,19.25,322.50,1.00,403.75,2.50,0.4050,-6.0925,4.8050
11301,2018,31,13,3,2018,12,20,6,6,25,...,0.25,18.50,23.75,315.25,1.50,429.00,2.00,0.2725,-9.8900,4.3275
11302,2018,31,14,9,2018,12,13,6,7,23,...,0.25,18.25,21.25,282.50,1.50,415.25,2.00,-3.1525,-5.2000,4.3975
11303,2018,31,15,16,2018,12,13,7,7,14,...,0.00,18.25,30.50,283.00,2.25,390.50,1.25,-8.4575,-7.1900,4.3475
11304,2018,31,16,22,2018,12,16,7,8,30,...,0.25,17.00,28.00,274.75,1.75,358.50,1.00,-7.5850,-4.3425,2.0925
11305,2018,31,17,30,2018,12,16,7,9,25,...,0.25,15.25,26.50,265.00,1.50,330.25,1.00,-7.7050,-4.2350,3.4925
11306,2019,31,5,6,2019,10,13,0,5,21,...,0.00,16.50,29.50,296.25,2.25,399.25,1.50,-4.6150,-12.7100,2.8075
11307,2019,31,6,13,2019,10,13,1,5,19,...,0.00,11.50,29.75,252.50,2.75,400.75,1.75,-12.8300,-10.5975,4.6025


### SweetViz 

In [56]:
df_report = sv.analyze(df_rolling)
#df_report.show_html()
#df_report.show_notebook()

/opt/anaconda3/lib/python3.8/site-packages/sweetviz/dataframe_report.py:74: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  all_source_names = [cur_name for cur_name, cur_series in source_df.iteritems()]
/opt/anaconda3/lib/python3.8/site-packages/sweetviz/dataframe_report.py:109: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  filtered_series_names_in_source = [cur_name for cur_name, cur_series in source_df.iteritems()


/opt/anaconda3/lib/python3.8/site-packages/sweetviz/series_analyzer_numeric.py:25: FutureWarning: The 'mad' method is deprecated and will be removed in a future version. To compute the same result, you may do `(df - df.mean()).abs().mean()`.
  stats["mad"] = series.mad()
/opt/anaconda3/lib/python3.8/site-packages/sweetviz/series_analyzer_numeric.py:25: FutureWarning: The 'mad' method is deprecated and will be removed in a future version. To compute the same result, you may do `(df - df.mean()).abs().mean()`.
  stats["mad"] = series.mad()
/opt/anaconda3/lib/python3.8/site-packages/sweetviz/series_analyzer_numeric.py:25: FutureWarning: The 'mad' method is deprecated and will be removed in a future version. To compute the same result, you may do `(df - df.mean()).abs().mean()`.
  stats["mad"] = series.mad()
/opt/anaconda3/lib/python3.8/site-packages/sweetviz/series_analyzer_numeric.py:25: FutureWarning: The 'mad' method is deprecated and will be removed in a future version. To compute the

In [57]:
# compares two dataframes via sweetviz
#df_comp = sv.compare(df, df_rolling)
#df_comp.show_notebook()

In [58]:
# another way to get great insights is to use the comparison functionality to split your dataset into 2 sub-populations.
#df_comp_intra = sv.compare_intra(df_rolling)

In [59]:
# using pandas to convert the prepared dataframe into a csv file that is model ready.
df_rolling.to_csv("Data/transformed.csv", index=False)